In [11]:
from client_side import *

config = connect_db(db_type='postgres', host='dbeda-client', database='test_cli', user='postgres', password='postgres', port='5432', interval=10)

Configuration data sent successfully.


In [70]:
#load prediction 데이터 잘 저장되고 잘 불러왓는지 테스트


data = query_performance_data(config, table='os_metric', metrics='cpu_percent', task='load prediction', recent_time_window='1 day')
df = pd.DataFrame(data['task'])

df

query_performance_data {'db_type': 'postgres', 'db_host': 'eda-client', 'db_name': 'test_cli', 'db_user': 'postgres', 'db_password': 'postgres', 'db_port': '5432', 'interval': 10}
{'table': 'os_metric', 'metric': 'cpu_percent', 'recent_time_window': '1 day', 'task': 'load prediction', 'config': {'db_type': 'postgres', 'db_host': 'eda-client', 'db_name': 'test_cli', 'db_user': 'postgres', 'db_password': 'postgres', 'db_port': '5432', 'interval': 10}}


,timestamp,predicted
0,2023-05-19 05:08:30,4.286743
1,2023-05-19 05:08:40,4.284088
2,2023-05-19 05:08:50,4.284088
3,2023-05-19 05:09:00,4.284088
4,2023-05-19 05:09:10,4.284088
...,...,...
175,2023-05-19 05:37:40,4.284088
176,2023-05-19 05:37:50,4.284088
177,2023-05-19 05:38:00,4.284088
178,2023-05-19 05:38:10,4.284088


In [133]:
visualize(config)

Row(sizing_mode='stretch_width')
    [0] Column(width=350)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Accordion(sizing_mode='stretch_width')
            [0] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [1] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=           column  ...)
            [2] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [3] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [4] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [5] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [6] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [7] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=             c...)
            [8] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [9] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [10] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=        column  ...)
            [11] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=              ...)
            [12] Tabulator(formatters={'key': BooleanFormatter(i...}, name="<i class='fa f..., show_index=False, sizing_mode='stretch_width', value=            column  ...)
    [1] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Row(sizing_mode='stretch_width')
            [0] TextInput(name='Title', placeholder='Enter a string f..., width=300)
            [1] Select(name='Tables', options=['performance', ...], value='performance', width=300)
            [2] Column(sizing_mode='stretch_width')
                [0] Select(name='Time range', options=['30 minutes', ...], value='30 minutes', width=300)
                [1] DatetimeRangePicker(disabled=True, value=(datetime.datetime(2023, ..., width=300)
            [3] Select(name='Auto refresh', options={'Off': 'None', ...}, value='None', width=300)
        [2] Card(title='Task', width=1200)
            [0] Row(sizing_mode='stretch_width')
                [0] Column(css_classes=['task_box'], sizing_mode='stretch_width')
                    [0] Row(sizing_mode='stretch_width')
                        [0] Select(name='Task', options=['Choose a task', ...], value='Choose a task', width=300)
                    [1] Row(sizing_mode='stretch_width')
                    [2] Row(sizing_mode='stretch_width')
                [1] Button(name='Add task', width=100)
        [3] Card(title='Split', width=1200)
            [0] Row(sizing_mode='stretch_width')
                [0] Select(name='Split', options=['None', 'column', ...], value='None', width=300)
                [1] Select(name='Split basis', width=300)
        [4] Row(sizing_mode='stretch_width')
            [0] Button(name='Draw', width=100)
            [1] Button(name='Clean', width=100)

In [12]:
collect_performance_data(config)

Performance data is being collected.


{'db_type': 'postgres',
 'db_host': 'dbeda-client',
 'db_name': 'test_cli',
 'db_user': 'postgres',
 'db_password': 'postgres',
 'db_port': '5432',
 'interval': 10}

In [129]:
# data loading
# data = query_performance_data(config, table='performance', metrics='tps', task='metrics', recent_time_window='4 hours')

data = query_performance_data(config, table='performance', metrics='tps', task='load prediction', start_time = "2023-05-23 00:00:00", end_time = "2023-05-23 12:00:00")

# "%Y-%m-%d %H:%M:%S"
df_metric = pd.DataFrame(data['metric'])
df_task = pd.DataFrame(data['task'])

df_task

2023-05-23 00:00:00
query_performance_data {'db_type': 'postgres', 'db_host': 'eda-client', 'db_name': 'test_cli', 'db_user': 'postgres', 'db_password': 'postgres', 'db_port': '5432', 'interval': 10}
{'table': 'performance', 'metric': 'tps', 'recent_time_window': 'Custom', 'task': 'load prediction', 'config': {'db_type': 'postgres', 'db_host': 'eda-client', 'db_name': 'test_cli', 'db_user': 'postgres', 'db_password': 'postgres', 'db_port': '5432', 'interval': 10}, 'start_time': '2023-05-23 00:00:00', 'end_time': '2023-05-23 12:00:00'}


,timestamp,predicted,lower_bound,upper_bound,analysis_time


In [98]:
split_date = df['timestamp'][int(len(df) * 0.7)]
train_df = df[df['timestamp'] <= split_date]
test_df = df[df['timestamp'] > split_date]

train_df

,timestamp,tps
0,2023-05-23 11:05:40,11.832079
1,2023-05-23 11:05:50,11.560025
2,2023-05-23 11:06:00,12.971010
3,2023-05-23 11:06:10,12.738367
4,2023-05-23 11:06:20,11.638327
...,...,...
945,2023-05-23 13:43:10,10.702627
946,2023-05-23 13:43:20,11.445576
947,2023-05-23 13:43:30,12.604301
948,2023-05-23 13:43:40,13.103584


## Load Prediction

In [99]:
# load prediction train
#response = train(config,train_df,'load prediction', pipeline = 'RNN')
response = train(config,train_df,'load prediction', pipeline = 'TCN')

<Response [200]>


In [100]:
get_trained_model(config, 'load prediction')

darts_RNN_20230519_154420.pickle
darts_TCN_20230523_150814.pickle
darts_ARIMA_20230523_145145.pickle


In [103]:
predicted = predict(config, 'load prediction', metric = 'tps', path="darts_TCN_20230523_150814.pickle")

In [104]:
predicted

,timestamp,predicted
0,2023-05-23 13:44:00,11.356294
1,2023-05-23 13:44:10,12.255639
2,2023-05-23 13:44:20,11.409054
3,2023-05-23 13:44:30,11.839214
4,2023-05-23 13:44:40,11.385554
...,...,...
175,2023-05-23 14:13:10,11.143083
176,2023-05-23 14:13:20,11.145175
177,2023-05-23 14:13:30,11.169814
178,2023-05-23 14:13:40,11.172859


## Anomaly Detection

In [105]:
response = train(config,train_df,'anomaly detection', pipeline = 'pyod')

<Response [200]>


In [106]:
get_trained_model(config, 'anomaly detection')

darts_kmeans_scorer_20230515_233712.pickle
darts_kmeans_scorer_20230515_202955.pickle
darts_pyod_20230523_155341.pickle
darts_kmeans_scorer_20230516_040138.pickle
darts_kmeans_scorer_20230515_172657.pickle


In [114]:
anom_score = predict(config, data = test_df, task = 'anomaly detection', path ="darts_pyod_20230523_155341.pickle")

In [115]:
anom_score

,timestamp,anomaly_score
0,2023-05-23 13:44:00,0.127975
1,2023-05-23 13:44:10,0.009582
2,2023-05-23 13:44:20,0.002268
3,2023-05-23 13:44:30,0.024345
4,2023-05-23 13:44:40,0.016807
...,...,...
401,2023-05-23 14:50:50,0.059468
402,2023-05-23 14:51:00,0.009982
403,2023-05-23 14:51:10,0.014458
404,2023-05-23 14:51:20,0.020490
